## ロジスティック回帰

- 目的変数が（基本的には）2値の質的データ
- 回帰という名前がついているが、分類を行う

In [ ]:
import pandas as pd
import seaborn as sns

# Iris データセット
iris = sns.load_dataset('iris')

# 2値データへの分類のため、species から setosa を除外
df = iris.query('species!="setosa"')
# versicolor, virginica だけになっていることを確認
# - 0, 50 行目を表示
df.iloc[[0, 50]]

In [ ]:
# 散布図でデータの重なりを見る
sns.pairplot(df, hue='species')

上記の散布図で versicolor, virginica の重なり具合を見ると：

- sepal_width は、かなりの重なりがある
- petal_width は、多少の重なりがある

<hr>

### ダミー変数

species には versicolor, virginica のようなカテゴリを表す値が入っている。
これをそのまま分類の予測対象（目的変数）とすることは、アルゴリズムによってはできない。
そのため、カテゴリを数値化することをダミー変数化という。

versicolor かどうかで 0, 1, virginica かどうかで 0, 1 という表現の仕方 (one-hot encoding) もあるが、この場合は virginica でなければ versicolor であることは明らかであるため、virginica かどうか 0, 1 と表現できれば十分になる。

In [ ]:
# speciesを 0, 1 にするためにダミー変数化
df = pd.get_dummies(data=df, drop_first=True)
# 確認のため 0, 50 行目を表示
df.iloc[[0, 50]]

<hr>
<img src='slides/6_20.png'>
<hr>

### 一番重なりが少なそうな petal_width を説明変数としてロジスティック回帰モデルを作成する

In [ ]:
from sklearn.linear_model import LogisticRegression

# 説明変数、目的変数の設定
X = df[['petal_width']]
Y = df.species_virginica

# ロジスティック回帰モデルを作成
# - C: 正則化のパラメータ（sklearnのロジスティック回帰はL2正則化がデフォルト）
#      Cを大きくして正則化の影響が小さくする
model = LogisticRegression(C=10000.0)
# 学習
model.fit(X, Y)

<hr>
<img src='slides/6_27.png'>
<hr>

In [ ]:
# 正確度、適合度、再現率（感度）、特異度
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

# 教師データについてモデルからの予測値を計算
Y_predict = model.predict(X)

# 混合行列から特異度を計算
matrix = confusion_matrix(Y, Y_predict)
specificity = matrix[0, 0] / (matrix[0, 1] + matrix[0, 0])

print('正確度: {:.3f}, 適合度: {:.3f}, 再現率: {:.3f}, 特異度: {:.3f}'.format(
    accuracy_score(Y, Y_predict), precision_score(Y, Y_predict),
    recall_score(Y, Y_predict), specificity))

<hr>
<img src='slides/6_29.png'>
<hr>
<img src='slides/6_33.png'>
<hr>

## 日本語フォントの設定

matplotlib のデフォルトでは英語フォントの設定になっているため、日本語を表示させたい場合には、日本語のフォントを設定する必要がある

In [ ]:
# matplotlib: 日本語フォントの設定
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Hiragino Maru Gothic Pro', 'Yu Gothic', 'Meirio', 
                               'Takao', 'IPAexGothic', 'IPAPGothic', 'Noto Sans CJK JP']

<hr>

In [ ]:
# ROC曲線, AUC
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# 分類確率からROC曲線、AUCを計算
Y_proba = model.predict_proba(X)
fpr, tpr, thresholds = roc_curve(Y, Y_proba[:, 1])
_auc = auc(fpr, tpr)

# ROC曲線の描画
plt.plot(fpr, tpr, label='ROC curve')
plt.xlabel('偽陽性率 (FP率)')
plt.ylabel('真陽性率 (TP率)')
plt.legend()
plt.title('ROC曲線: AUC={:.3f}'.format(_auc))

<hr>

### ロジスティック回帰モデルの精度の比較

説明変数

1. 一番重なりが少なそうな petal_width だけ
2. かなりの重なりがある petal_width だけ
3. 全部 (sepal_length, sepal_width, petal_length, petal_width)

In [ ]:
# 説明変数のリストと目的変数の設定
X_labels_list = [
    ['petal_width'],
    ['sepal_width'],
    ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
]
Y = df.species_virginica

# ロジスティック回帰モデル
model = LogisticRegression(C=10000.0)

for i in range(0, len(X_labels_list)):
    # ラベルから説明変数の設定
    X_labels = X_labels_list[i]
    X = df[X_labels]
    # 学習
    model.fit(X, Y)
    # 教師データについてモデルからの予測値を計算
    Y_predict = model.predict(X)

    # 特異度の計算
    matrix = confusion_matrix(Y, Y_predict)
    specificity = matrix[0, 0] / (matrix[0, 1] + matrix[0, 0])
    # 精度
    print('{}: {}'.format(i, X_labels))
    print('正確度: {:.3f}, 適合度: {:.3f}, 再現率: {:.3f}, 特異度: {:.3f}'.format(
        accuracy_score(Y, Y_predict), precision_score(Y, Y_predict),
        recall_score(Y, Y_predict), specificity))
    # ROC, AUC
    Y_proba = model.predict_proba(X)
    fpr, tpr, thresholds = roc_curve(Y, Y_proba[:, 1])
    plt.plot(fpr, tpr, label='{}: AUC={:.3f}'.format(i, auc(fpr, tpr)))

plt.xlabel('偽陽性率 (FP率)')
plt.ylabel('真陽性率 (TP率)')
plt.title('ROC曲線')
plt.legend()
plt.show()